# Tradesman

Tradesman is a friendly model builder for transportation models. 

On this example, we show how to create a transportation model for Gibraltar, a small british territory in mainland Europe. In the end of the presetation, we also show some data visualization.

## Running on Google Colab

A few steps need to be taken after pressing the button below

<a href="https://colab.research.google.com/github/outerl/tradesman-demo/blob/main/model_building_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

1. Uncomment and run cell below
2. re-start your runtime (Runtime-Restart Runtime or ctrl+m)
3. Start again, but from the **Import package** section

In [1]:
# !apt-get update && apt-get install libsqlite3-mod-spatialite
# !apt-get install -y libspatialite-dev
# !pip install "numpy>1.22,<1.24" --upgrade
# !pip install tradesman

## Import package

In [ ]:
import os
os.environ['USE_PYGEOS'] = '0'

from uuid import uuid4
from tradesman.model import Tradesman


## Create model

We're going to save our model in the *temp* folder

If you have are running this notebook on Google Colab and want to save your model into your Google Drive, you should:

1. Un-comment and run the cell below
2. Accept the terms and conditions, and replace `"gettempdir()"` with `"/content/gdrive"`.


In [3]:
# from google.colab import drive
# drive.mount("/content/gdrive")

In [4]:
from tempfile import gettempdir

model_place = "Gibraltar"
folder = os.path.join(gettempdir(), uuid4().hex)

In [5]:
model = Tradesman(network_path=folder, model_place=model_place)

We could just create the model with default parameters by calling  `model.create()`, but we want to make all components explicit

In [6]:
%%time
# Import the model area
model.import_model_area()

# The borders for the country that includes this modeling area
model.add_country_borders()

# Imports the political subdivisions for this region
model.import_subdivisions(subdivision_levels=2, overwrite=True)

CPU times: total: 31.2 ms
Wall time: 1.42 s


In [7]:
%%time
# Import the model network
model.import_network()

CPU times: total: 1.2 s
Wall time: 12.5 s


In [8]:
%%time
# Import population
model.import_population()

CPU times: total: 62.5 ms
Wall time: 236 ms


As Monaco has a really small territorial area, we'll create Traffic Analysis Zones (TAZs) more suitable sizes to the country rather than the default ones.

In [9]:
# Build TAZs with zones in the range of 100 to 500 people
# The default range is 500 to 10,000
# The `hexbin_size` parameter is the size of the hexbin side in meters.  Very small hexbins make the computation much slower, but may be necessary if you population targets are small.  The default parameter is 200
model.build_zoning(hexbin_size=100, min_zone_pop=100, max_zone_pop=500)

Expect 2,479 total hexbins for this bounding box


100%|██████████| 37/37 [00:00<00:00, 875.76it/s]
c:\src\Tradesman-demo\.venv\lib\site-packages\tradesman\model_creation\zoning\zones_with_location.py:23: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = gpd.GeoDataFrame(hexb[["hex_id"]], geometry=hexb.centroid, crs="EPSG:4326")
c:\src\Tradesman-demo\.venv\lib\site-packages\geopandas\array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
c:\src\Tradesman-demo\.venv\lib\site-packages\tradesman\model_creation\zoning\create_clusters.py:33: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centro

Done 0/1 states
Queue for analysis: 27 (Done: 43 (341 people/zone))


1it [00:00,  1.22it/s]
c:\src\Tradesman-demo\.venv\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
  warnings.warn(message)
c:\src\Tradesman-demo\.venv\lib\site-packages\geopandas\geodataframe.py:1475: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
c:\src\Tradesman-demo\.venv\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
  warnings.warn(message)
c:\src\Tradesman-demo\.venv\lib\site-packages\geopandas\geodataframe.py:1475: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


In [10]:
%%time
# Imports the population pyramid for our new zone system
model.import_pop_by_sex_and_age()

CPU times: total: 1.61 s
Wall time: 5.08 s


In [11]:
%%time
# Import amenities from Open-Street Maps
model.import_amenities()

CPU times: total: 78.1 ms
Wall time: 440 ms


In [12]:
# Import buildings
model.import_buildings(download_from_bing=False)

## Create a synthetic population

Let's create our synthetic population for Gibraltar! It's a two-step process: first, we sample a population dataset to create our seed sample that resembles Gibraltar (seed creation) and then, we run the synthesizer.
The sample that ships with the software is a sample for Texas, and the software is separated in the creation of the PopulationSim configuration and the actual population generation to allow users to use a more appropriate sample if available

In [13]:
# Create sample to build synthetic population
model.build_population_synthesizer_data(sample_size=0.02)

Calculated vs Expected percentage of households per size in the sample

# Person      Calculated    Expected
----------  ------------  ----------
1 person           20.08       19.06
2-3 person         52.46       53.4
4-5 person         26.64       25.56
6+ person           0.82        1.98

Difference in calculated vs expected percentage of households in the sample

# Person      Diff.
----------  -------
1 person       1.02
2-3 person    -0.94
4-5 person     1.08
6+ person     -1.16



From the output, we can visualize if there are more or fewer households of certain categories. It's important to highlight that differences between expected and found values are expected because the data used to create the sample doesn't belong to Monaco.

In [14]:
# Create synthetic population
model.synthesize_population()

c:\src\Tradesman-demo\.venv\lib\site-packages\tradesman\model_creation\synthetic_population\create_synthetic_population.py:108: UserWarning: Synthetic population was not generated. Please increase your sample size or change the number of threads.
  warnings.warn(


After the execution of the above cell, there are two main outputs. The ones related to non-controlled or controlled variables. The first one represents the differences between expected and found values for non-controlled variables based on United Nation Household survey data. The graphs represent the percentage difference between the synthetic over the expected population for controlled variables.

## And as promised, a bit of data visualization

### Import libraries

In [15]:
import geopandas as gpd
import folium

### Layer set up

In [16]:
colors = ["#219EBC", "#ffb703", "#8ECAE6", "#023047", "#fb8500"]
location = [36.142260, -5.346601] # This is the location of the Monte Carlo Casino, in case you're curious

In [17]:
# Let's create a variable with our project database connection
cnx = model._project.conn

### Plot the network

In [18]:
query = "SELECT link_type, distance, modes, ST_AsBinary(geometry) geom FROM links;"
links = gpd.read_postgis(query, con=cnx, geom_col="geom", crs=4326)

# We'll plot only some link types
links = links[links.link_type.isin(["residential", "primary", "secondary", "tertiary"])]

In [19]:
m = None

for idx, tp in enumerate(links.link_type.unique()):
  gdf = links[links.link_type == tp]
  if m:
    gdf.explore(m=m, name=tp, tiles="CartoDB positron", tooltip=False, popup=True,
                  zoom_start=14, location=location, legend=False, color=colors[idx])
  else:
    m = gdf.explore(name=tp, tiles="CartoDB positron", tooltip=False, popup=True,
                      zoom_start=14, location=location, legend=False, color=colors[idx])

folium.LayerControl().add_to(m)

m

### Plot the populational density


In [20]:
query = "SELECT *, ST_AsBinary(geometry) geom FROM zones;"
zones = gpd.read_postgis(query, con=cnx, geom_col="geom", crs=4326)
zones.drop(columns=["geometry"], inplace=True)

# Let's create our populational densisity variable
zones["POP_DENSITY"] = zones["population"] / (zones["geom"].to_crs(3857).area * 10e-6)


In [21]:
zones.explore("POP_DENSITY", tiles="CartoDB positron", cmap="Blues", tooltip=False,
              style_kwds={"fillOpacity": 1.0}, zoom_start=14, location=location, popup=True)


In [22]:
model.close()